In [242]:
from pysmiles import read_smiles
import networkx as nx
import rdkit
from rdkit.Chem import MolFromSmiles
from rdkit import Chem
from spektral.data import Dataset, DisjointLoader, Graph
import numpy as np
import random
import pandas as pd

In [243]:
pp = pd.read_csv('../data/split.csv', sep = ";")
pp = pp[["p_np", "smile"]]
pp.head()

,p_np,smile
0,0,CC(C)(Oc1ccc(Cl)cc1)C(O)=O
1,1,Cn1c2CCC(Cn3ccnc3C)C(=O)c2c4ccccc14
2,1,C1=CC=CC3=C1C(C2=CC=CC=C2)(SC3(C)C)CCCNC
3,0,CC(=O)OCC1=C(N2[C@H](SC1)[C@H](NC(=O)CSc3ccncc...
4,1,CN(C)CCCN1c2ccccc2Sc3ccc(cc13)C(F)(F)F


In [244]:
permitted_list = ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe', 'I']

def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """
    if x not in permitted_list:
        x = permitted_list[-1]
    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]
    return binary_encoding
    
def getAtomFeatures(atom):
     # compute atom features
    
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list)
    
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    
    is_in_a_ring_enc = [int(atom.IsInRing())]
    
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    
    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]
    
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]
    
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]

    atom_features = np.concatenate([atom_type_enc, n_heavy_neighbors_enc, formal_charge_enc, hybridisation_type_enc, is_in_a_ring_enc, is_aromatic_enc, atomic_mass_scaled, vdw_radius_scaled, covalent_radius_scaled])
    return atom_features

def getNodeFeatures(nodevec):
    return np.array([getAtomFeatures(atom) for atom in nodevec.GetAtoms()])
        
    

In [245]:
tempList = []

##this takes forever i'm sorry

for index, row in pp.iterrows():
    smile = row["smile"]
    mol = MolFromSmiles(smile)
    #nx.convert_node_labels_to_integers(mol)
    #nodevec = list(mol.nodes(data='element'))
    molFeatures = getNodeFeatures(mol)
    adj = rdkit.Chem.rdmolops.GetAdjacencyMatrix(mol)
    
    g = Graph(a = adj, x = molFeatures, y = int(row["p_np"]))
    tempList.append(g)


[22:12:26] WARNING: not removing hydrogen atom without neighbors
[22:12:26] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not removing hydrogen atom without neighbors
[22:12:27] WARNING: not r

In [246]:
pp["graph"] = tempList

graphs = pp["graph"]
graphs


0      Graph(n_nodes=14, n_node_features=40, n_edge_f...
1      Graph(n_nodes=22, n_node_features=40, n_edge_f...
2      Graph(n_nodes=22, n_node_features=40, n_edge_f...
3      Graph(n_nodes=28, n_node_features=40, n_edge_f...
4      Graph(n_nodes=24, n_node_features=40, n_edge_f...
                             ...                        
847    Graph(n_nodes=20, n_node_features=40, n_edge_f...
848    Graph(n_nodes=15, n_node_features=40, n_edge_f...
849    Graph(n_nodes=36, n_node_features=40, n_edge_f...
850    Graph(n_nodes=20, n_node_features=40, n_edge_f...
851    Graph(n_nodes=33, n_node_features=40, n_edge_f...
Name: graph, Length: 852, dtype: object

In [247]:
class MyDataset(Dataset):
    """
    load our data
    """
    def __init__(self, graphs, **kwargs):
        self.graphs = graphs
        super().__init__(**kwargs)
    

        
    def read(self):
        # We must return a list of Graph objects
        return self.graphs


In [248]:
dataset = MyDataset(graphs = graphs)

In [249]:
dataset.filter(lambda g: g.n_nodes < 45)

In [250]:
max_degree = dataset.map(lambda g: g.a.sum(-1).max(), reduce=max)
max_degree

4

In [251]:
from spektral.transforms import Degree

dataset.apply(Degree(max_degree))

In [252]:
from spektral.transforms import GCNFilter

dataset.apply(GCNFilter())

In [253]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.3)
        self.dense1 = Dense(n_hidden, 'sigmoid')
        self.dense2 = Dense(n_hidden/2, 'sigmoid')
        self.dense = Dense(n_labels, 'sigmoid')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dropout(out)
        out = self.dense1(out)
        out = self.dropout(out)
        out = self.dense2(out)
        out = self.dropout(out)
        out = self.dense(out)

        return out


In [254]:
from keras.optimizers import Adam

model = MyFirstGNN(100, dataset.n_labels)
opt = Adam(learning_rate=5e-5)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [255]:
from spektral.data import BatchLoader

np.random.shuffle(dataset)
split = int(0.8 * len(dataset))
data_tr, data_te = dataset[:split], dataset[split:]

loader = BatchLoader(data_tr)

/var/folders/xf/mwvkzqj51j93w8hrtltgvgn40000gn/T/ipykernel_20316/4288830762.py:3: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset)


In [256]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=300)

Epoch 1/300


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
2023-12-04 22:12:28.635274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


678/678 [==============================] - 2s 2ms/step - loss: 0.7397 - accuracy: 0.5029
Epoch 2/300
678/678 [==============================] - 1s 2ms/step - loss: 0.7106 - accuracy: 0.5339
Epoch 3/300
678/678 [==============================] - 1s 2ms/step - loss: 0.7026 - accuracy: 0.5324
Epoch 4/300
678/678 [==============================] - 1s 2ms/step - loss: 0.7077 - accuracy: 0.5251
Epoch 5/300
678/678 [==============================] - 1s 2ms/step - loss: 0.6904 - accuracy: 0.5737
Epoch 6/300
678/678 [==============================] - 1s 2ms/step - loss: 0.6888 - accuracy: 0.5501
Epoch 7/300
678/678 [==============================] - 1s 2ms/step - loss: 0.6812 - accuracy: 0.5634
Epoch 8/300
678/678 [==============================] - 1s 2ms/step - loss: 0.6680 - accuracy: 0.5855
Epoch 9/300
678/678 [==============================] - 1s 2ms/step - loss: 0.6586 - accuracy: 0.6121
Epoch 10/300
678/678 [==============================] - 1s 2ms/step - loss: 0.6489 - accuracy: 0.6268
E

In [ ]:
loader2 = BatchLoader(data_te)
loss = model.evaluate(loader2.load(), steps=loader2.steps_per_epoch)


#print('Test loss: {}'.format(loss))

 73/170 [===========>..................] - ETA: 0s - loss: 0.5311 - accuracy: 0.7808

2023-12-04 22:08:09.771688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


170/170 [==============================] - 0s 2ms/step - loss: 0.4647 - accuracy: 0.8000
